参考：https://blog.csdn.net/len9596/article/details/79990823

# 简单装饰器

In [70]:
def use_logging(func):
    def wrapper(*args,**kwargs):
        logging.warning("%s is running"% func.__name__)
        return func(*args,**kwargs)
    wrapper.__doc__ = func.__doc__
    return wrapper


def bar():
    """测试"""
    print('I am bar')

bar = use_logging(bar)
bar()
print(bar.__name__)
print(bar.__doc__)

I am bar
wrapper
测试


函数use_logging就是装饰器，它把执行真正业务方法的func包裹在函数里面，看起来像bar被use_logging装饰了。在这个例子中，函数进入和退出时 ，被称为一个**横切面(Aspect)**，这种编程方式被称为**面向切面的编程(Aspect-Oriented Programming)。**

@符号可以理解为避免再一次赋值操作的 **简写** 

```
@ 等价于 a = d(a)
```

这样我们就可以省去bar = use_logging(bar)这一句了，直接调用bar()即可得到想要的结果。

注意：
1. 原函数的一些元属性会丢失，比如: 文档字符串`__doc__`，需要重新赋值给内部函数。

2. 参数要通过`(*args,**kwargs)` 全部传递过去。

3. 内部函数的返回值，是对被装饰函数简单调用就结束，还是 `return` 其返回值。

In [73]:
@use_logging
def bar(param=2):
    print('param is: {}'.format(param))
    return 'I am bar'


print(bar(param=3))
print(bar.__name__)

param is 3
I am bar
wrapper


# 带参数

在上面的装饰器调用中，比如@use_logging，该装饰器唯一的参数就是执行业务的函数。装饰器的语法允许我们在调用时，提供其它参数，比如
```
@decorator(a)
```
这样，就为装饰器的编写和使用提供了更大的灵活性。

在`decorator(a)`阶段 ,就已经发生了一次函数调用，并返回了像上面所说的简单装饰器函数use_logging，所以`decorator` 函数具体实现方法，就是在简单装饰器上再包装一层函数，将简单装饰器作为返回值。

In [76]:
import logging


def use_logging(level):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if level == "warn":
                logging.warning("%s is running" % func.__name__)
            return func(*args, **kwargs)

        return wrapper
    return decorator



# 不用@ 符号写法
def foo(name='foo'):
    print('I am bar')

foo = use_logging('warn')(foo)
foo()

I am bar


In [77]:
@use_logging("warn")
def foo(name='foo'):
    print('I am bar')

foo()
print(foo.__name__)

I am bar
wrapper


# 类装饰器

相比函数装饰器，类装饰器具有灵活度大、高内聚、封装性等优点。使用类装饰器还可以依靠类内部的`__call__`方法，当使用 @ 形式将装饰器附加到函数上时，就会调用此方法。

In [89]:
class Foo(object):
    def __init__(self, func):
        self._func = func
        self.__doc__ = func.__doc__

    def __call__(self):
        print('class decorator runing')
        self._func()
        print('class decorator ending')


@Foo
def bar():
    """test doc"""
    print('bar')


bar()
print(bar.__doc__)

class decorator runing
bar
class decorator ending
test doc


同样的，原函数的元信息不见了，比如函数的docstring、name、参数列表，需要手动传递进去。

# 内置装饰器

- @staticmathod
- @classmethod
- @property

# 装饰器的运行顺序: 从下往上
```py
@a
@b
@c
def f ():
```
等效于

```py
f = a(b(c(f)))
```